In [ ]:
# 02_ingestion_usgs.ipynb
from pyspark.sql import SparkSession
import requests, json, time, os
from datetime import datetime

# Initialiser la session Spark
spark = SparkSession.builder.appName("USGS_Ingestion").getOrCreate()

# Définir le chemin HDFS - ESSAYER CES OPTIONS DANS L'ORDRE :
# Option 1: localhost (si HDFS est en local)
RAW_BASE = "hdfs://localhost:9000/raw/usgs"

# Option 2: Adresse IP directe (si vous connaissez l'IP de namenode)
# RAW_BASE = "hdfs://192.168.1.100:8020/raw/usgs"

# Option 3: file:// pour système de fichiers local (fallback)
# RAW_BASE = "file:///tmp/raw/usgs"

# Créer le répertoire temporaire local dans le répertoire utilisateur actuel
LOCAL_TEMP_DIR = "./usgs_temp"
os.makedirs(LOCAL_TEMP_DIR, exist_ok=True)

def test_hdfs_connection():
    """Tester la connexion HDFS et trouver la bonne configuration"""
    test_paths = [
        "hdfs://localhost:9000",
        "hdfs://127.0.0.1:9000", 
        "hdfs://namenode:8020",
        "hdfs://0.0.0.0:9000",
        "file:///tmp"
    ]
    
    for path in test_paths:
        try:
            test_dir = f"{path}/test_connection"
            spark.sparkContext.parallelize([1]).saveAsTextFile(test_dir)
            print(f"✓ Connexion réussie: {path}")
            # Nettoyer
            fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
            fs.delete(spark._jvm.org.apache.hadoop.fs.Path(test_dir), True)
            return path
        except Exception as e:
            print(f"✗ Échec {path}: {str(e)[:100]}...")
    
    print("⚠ Aucune connexion HDFS trouvée, utilisation du système de fichiers local")
    return "file:///tmp"

def fetch_and_save(feed_url="https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"):
    """
    Récupère les données sismiques de l'USGS et les sauvegarde
    """
    try:
        # Récupérer les données depuis l'API USGS
        print("Récupération des données depuis l'API USGS...")
        r = requests.get(feed_url, timeout=30)
        r.raise_for_status()
        data = r.json()
        
        # Créer un fichier local avec timestamp
        timestamp = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
        local_path = f"{LOCAL_TEMP_DIR}/earthquakes_{timestamp}.json"
        
        # Sauvegarder les données en local temporairement
        with open(local_path, "w", encoding="utf-8") as f:
            json.dump(data, f)
        
        print(f"Données brutes sauvegardées localement: {local_path}")
        
        # Lire le JSON avec Spark
        print("Lecture des données avec Spark...")
        df = spark.read.json(local_path)
        
        # Vérifier si des données sont présentes
        if df.count() == 0:
            print("Aucune donnée à traiter")
            os.remove(local_path)
            return None
            
        # Normaliser la structure: features -> properties + geometry
        print("Transformation des données...")
        df2 = df.selectExpr("explode(features) as feat").selectExpr(
            "feat.properties.*",
            "feat.geometry.coordinates as coords",
            "feat.id as event_id"
        ).withColumnRenamed("time", "time_ms")
        
        # Convertir les colonnes
        import pyspark.sql.functions as F
        df2 = df2.withColumn("event_time", (F.col("time_ms")/1000).cast("timestamp")) \
                 .withColumn("longitude", F.col("coords").getItem(0)) \
                 .withColumn("latitude", F.col("coords").getItem(1)) \
                 .withColumn("depth_km", F.col("coords").getItem(2))
        
        # Afficher le schéma et un échantillon
        print("Schéma des données transformées:")
        df2.printSchema()
        
        event_count = df2.count()
        print(f"Nombre d'événements à sauvegarder: {event_count}")
        
        if event_count > 0:
            df2.show(3, truncate=False)
        
        # Créer le chemin de sortie avec partitionnement par date
        date_partition = datetime.utcnow().strftime('%Y/%m/%d')
        out_path = f"{RAW_BASE}/events/{date_partition}"
        
        # Sauvegarder en format Parquet
        print(f"Sauvegarde dans: {out_path}")
        df2.write.mode("append").parquet(out_path)
        print(f"✓ Données sauvegardées avec succès: {out_path}")
        
        # Nettoyer le fichier local temporaire
        os.remove(local_path)
        print(f"✓ Fichier local temporaire supprimé")
        
        return out_path
        
    except Exception as e:
        print(f"❌ Erreur lors de l'ingestion: {e}")
        # Essayer de sauvegarder localement en fallback
        try:
            if 'local_path' in locals():
                backup_path = f"{LOCAL_TEMP_DIR}/backup_earthquakes_{timestamp}.json"
                os.rename(local_path, backup_path)
                print(f"✓ Données sauvegardées en local: {backup_path}")
        except:
            pass
        return None

# Tester la connexion HDFS au démarrage
print("=== Test de connexion HDFS ===")
BASE_PATH = test_hdfs_connection()
RAW_BASE = f"{BASE_PATH}/raw/usgs"
print(f"Chemin HDFS utilisé: {RAW_BASE}")

# Point d'entrée principal
if __name__ == "__main__":
    print("\n=== Démarrage de l'ingestion USGS ===")
    
    for i in range(3):  # À ajuster/supprimer en production
        print(f"\n--- Itération {i+1}/3 ---")
        try:
            result = fetch_and_save()
            if result:
                print(f"✓ Itération {i+1}/3 terminée avec succès")
            else:
                print(f"⚠ Itération {i+1}/3 échouée ou aucune donnée")
                
        except Exception as e:
            print(f"❌ Erreur lors de l'itération {i+1}: {e}")
        
        # Pause de 10 secondes entre chaque extraction (pour tests)
        if i < 2:
            print("⏳ Attente de 10 secondes...")
            time.sleep(10)
    
    print("\n=== Ingestion USGS terminée ===")